In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.read_csv('/home/foodstruct_nutritional_facts - foodstruct_nutritional_facts.csv')
df.head(5)

df_filtered = df[df['Calcium'].notnull()]
df_filtered

,Food Name,Category Name,Calcium,Calories,Carbs,Cholesterol,Copper,Fats,Fiber,Folate,...,Vitamin D,Vitamin E,Vitamin K,Omega-3 - ALA,Omega-6 - Eicosadienoic acid,Omega-6 - Gamma-linoleic acid,Omega-3 - Eicosatrienoic acid,Omega-6 - Dihomo-gamma-linoleic acid,Omega-6 - Linoleic acid,Omega-6 - Arachidonic acid
0,Acerola,Fruits,12.0,32.0,NaN,0.0,9.0,3.0,11.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple,Fruits,6.0,52.0,NaN,0.0,3.0,17.0,24.0,3.0,...,0.0,18.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apricot,Fruits,13.0,48.0,NaN,0.0,8.0,39.0,2.0,9.0,...,0.0,89.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dried fruit,Fruits,55.0,241.0,NaN,0.0,34.0,51.0,73.0,10.0,...,0.0,43.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Avocado,Fruits,12.0,160.0,NaN,0.0,19.0,15.0,67.0,81.0,...,0.0,21.0,21.0,11.0,0.0,2.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166,Sesame chicken,"Meals, Entrees, and Side Dishes",12.0,293.0,27.0,59.0,5.0,14.0,7.0,8.0,...,1.0,13.0,27.0,77.0,1.0,4.0,0.0,1.0,59.0,NaN
1167,Vermicelli,"Meals, Entrees, and Side Dishes",55.0,331.0,82.0,0.0,19.0,1.0,39.0,0.0,...,0.0,51.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168,Baby food,Baby Foods,733.0,379.0,73.0,0.0,33.0,44.0,75.0,43.0,...,0.0,37.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1169,Zwieback,Baby Foods,20.0,426.0,74.0,8.0,15.0,97.0,25.0,87.0,...,0.0,24.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [2]:
df1 = df.copy()
print("Tamanho DF original", len(df1))

columns = df1.columns.to_list()
columns_x = []
for column in columns:
  if column not in ['Food Name', 'Category Name']:
    if df1[column].isnull().sum()/len(df1) < 0.4:
      df1 = df1[df1[column].notnull()]
      columns_x.append(column)

print("Tamanho DF após filtro", len(df1))
print("# de features:", len(columns_x))

df1 = df1[columns_x]
df1 = df1.dropna()
print("Tamanho DF após remoção de nulos", len(df1))

x = df1.drop('Calories', axis=1)
y = df1['Calories']
model = LinearRegression()
model.fit(x, y)
print("R²", model.score(x, y))


Tamanho DF original 1171
Tamanho DF após filtro 398
# de features: 47
Tamanho DF após remoção de nulos 398
R² 0.5092120651795491


In [3]:
# nulls = df.isnull().sum().to_frame().sort_values(by=0)
# nulls

# rotina para remover nulos em colunas
def fix(target, df):
  # preservei meu dataset original
  df1 = df.copy()
  # derrubei as colunas que não devem participar do meu modelo
  df1.drop('Food Name', axis=1, inplace=True)
  df1 = df1.drop('Category Name', axis=1)
  df1.drop('Calories', axis=1, inplace=True)

  # para cada coluna existente no meu modelo, verifico se proporção de nulos
  # é superior a 10%; se sim, removo a coluna.
  # se não, filtro as linhas com observações nulas
  columns = df1.columns.to_list()
  df_filtered = df[df[target].isnull()]
  for column in columns:
    if df_filtered[column].isnull().sum()/len(df_filtered) < 0.1 or column == target:
      # somente filtra
      df1 = df1[df1[column].notnull()]
    else:
      # remove coluna
      df1.drop(column, axis=1, inplace=True)

  # separo minhas variáveis x e y
  y = df1[target].to_numpy()
  x = df1.drop(target, axis=1)
  x_columns = x.columns
  x = x.to_numpy()

  # gero o modelo de regressõa linear
  model = LinearRegression()
  model.fit(x, y)
  print("R²", model.score(x, y))

  # Calculo o valor sintético das variáveis nulas
  def complete(row):
    # se já tenho o valor da variável alvo, mantenho
    if not pd.isna(row[target]):
      return row[target]
    for column in x_columns:
      if pd.isna(row[column]):
        return row[target]

    # caso contrário, faço a previsão
    campos = row[x_columns]
    y_pred = round(model.predict([campos])[0])
    return y_pred
  # aplico a função de previsão para todas as linhas
  df[target] = df.apply(complete, axis=1)

fix('Copper', df)
fix('Fats', df)
fix('Fiber', df)



R² 0.035818937600876066
R² 0.09523363671796536
R² 0.17105837778836996


In [4]:
columns = df.columns.to_list()
columns.remove('Food Name')
columns.remove('Category Name')
columns.remove('Calories')

df1 = df.copy()
for column in columns:
  if df[column].isnull().sum()/len(df) < 0.3:
    df1 = df1[df1[column].notnull()]
  else:
    columns.remove(column)
print(len(df1))


714
